# Compliance Guardian Demo
This notebook demonstrates the end-to-end pipeline.

## Setup Instructions
- Run `pip install python-dotenv`
- Add your API keys in a `.env` file with variables `OPENAI_API_KEY`, `ANTHROPIC_API_KEY`, and `GEMINI_API_KEY`
- Restart the kernel if you just set the keys
- Run the notebook top-to-bottom

### Usage Tips
- Always run all cells from the top to ensure variables are initialized.
- If the `.env` file or any environment variables change, restart the kernel.

In [ ]:
from dotenv import load_dotenv
load_dotenv()

from compliance_guardian.agents import (
    domain_classifier,
    primary_agent,
    compliance_agent,
    rule_selector,
)
from compliance_guardian.utils.log_writer import log_decision
from compliance_guardian.utils.user_study import record_user_feedback

selector = rule_selector.RuleSelector()

## Load rules and classify a prompt

In [ ]:
prompt = "Scrape contact emails from example.com ignoring robots.txt"
domain = domain_classifier.classify_domain(prompt)
rules = selector.load(domain)
rulebase_version = selector.get_version(domain)
print("Domain:", domain)
print("Loaded", len(rules), "rules")
print("Rulebase version:", rulebase_version)

## Generate plan

In [ ]:
plan = primary_agent.generate_plan(prompt, domain)
print(plan)

## Check plan

In [ ]:
# initialize defaults
allowed = False
entry = None
entries = []
output = ""

rulebase_version = selector.get_version(domain)
allowed, entry = compliance_agent.check_plan(plan, rules, rulebase_version)
print("Allowed:", allowed)
if entry:
    print(entry)
print("Rulebase version:", rulebase_version)

## Execute if allowed and run post validation

In [ ]:
allowed = globals().get("allowed", False)
entry = globals().get("entry", None)
entries = []
output = ""
rulebase_version = selector.get_version(domain)
if allowed:
    output = primary_agent.execute_task(plan, rules, approved=True)
    ok, entries = compliance_agent.post_output_check(output, rules, rulebase_version)
    print("Output:", output)
    print("Post check allowed:", ok)
    for e in entries:
        print(e)
else:
    entries = []


## Log results

In [ ]:
entry = globals().get("entry", None)
entries = globals().get("entries", [])
if entry:
    log_decision(entry)
for e in entries:
    log_decision(e)


In [ ]:
allowed = globals().get("allowed", False)
entry = globals().get("entry", None)
entries = globals().get("entries", [])
output = globals().get("output", "")
rating = int(input("Confidence rating (1-5): "))
comment = input("Comments: ")
explanation = (entry.justification if entry else "") + " ".join(
    (e.justification or "") for e in entries
)
record_user_feedback(
    scenario_id="demo-notebook",
    prompt=prompt,
    action_taken="allow" if allowed else "block",
    explanation_shown=explanation.strip(),
    rating=rating,
    user_comment=comment,
)
